In [3]:
import os

import pandas as pd

# Specify the directory containing the JSON files
directory = './BitcoinData/'

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json'):  # Process only JSON files
        file_path = os.path.join(directory, filename)

        # Read the JSON file into a DataFrame
        df = pd.read_json(file_path)

        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

columns = [
    'Open Time',  # Timestamp for when the candlestick opened
    'Open Price',  # Price when the candlestick opened
    'High Price',  # Highest price during the interval
    'Low Price',  # Lowest price during the interval
    'Close Price',  # Price when the candlestick closed
    'Volume',  # Volume traded during the interval
    'Close Time',  # Timestamp for when the candlestick closed
    'Quote Asset Volume',  # Quote asset volume (e.g., USDT)
    'Number of Trades',  # Number of trades during the interval
    'Taker Buy Base Volume',  # Volume of the base asset (e.g., BTC) bought by takers
    'Taker Buy Quote Volume',  # Quote asset (e.g., USDT) spent by takers
    'Ignore'  # Ignored column (always 0)
]

# Assign these columns to your DataFrame
combined_df.columns = columns

# Optionally, if you want to convert the timestamps to human-readable format
combined_df['Open Time'] = pd.to_datetime(combined_df['Open Time'], unit='ms')
combined_df['Close Time'] = pd.to_datetime(combined_df['Close Time'], unit='ms')

combined_df['Day'] = combined_df['Open Time'].dt.day_name()

# Display the combined DataFrame
print(combined_df)

                 Open Time  Open Price  High Price  Low Price  Close Price  \
0      2019-09-08 17:55:00     10000.0     10000.0    10000.0      10000.0   
1      2019-09-08 18:00:00     10000.0     10000.0    10000.0      10000.0   
2      2019-09-08 18:05:00     10000.0     10000.0    10000.0      10000.0   
3      2019-09-08 18:10:00     10000.0     10000.0    10000.0      10000.0   
4      2019-09-08 18:15:00     10000.0     10000.0    10000.0      10000.0   
...                    ...         ...         ...        ...          ...   
539094 2024-10-22 16:10:00     67143.8     67225.2    67130.0      67196.0   
539095 2024-10-22 16:15:00     67196.0     67212.0    67023.5      67044.8   
539096 2024-10-22 16:20:00     67044.9     67079.3    66931.2      67028.9   
539097 2024-10-22 16:25:00     67028.9     67036.7    66935.1      66994.0   
539098 2024-10-22 16:30:00     66994.0     67105.7    66982.0      67082.5   

          Volume              Close Time  Quote Asset Volume  \

In [6]:
# Convert prices to float for comparisons
combined_df['Open Price'] = combined_df['Open Price'].astype(float)
combined_df['Close Price'] = combined_df['Close Price'].astype(float)

# Create a new column 'is_up' that checks if the Close Price is higher than the Open Price
combined_df['is_up'] = combined_df['Close Price'] > combined_df['Open Price']

# Group by week
combined_df['Week'] = combined_df['Open Time'].dt.to_period('W')

# Get the first 'm5' open price and last 'm5' close price for each week
first_of_week = combined_df.groupby('Week').first()
last_of_week = combined_df.groupby('Week').last()

# Compare the last close price with the first open price for each week
weekly_comparison = (last_of_week['Close Price'] > first_of_week['Open Price'])

# Map the result back to every row of the original dataframe, filling each week's 'is_weekly_up'
combined_df['is_weekly_up'] = combined_df['Week'].map(weekly_comparison)

# Keep only the columns for day name, time, and the two new columns
final_df = combined_df[['Day', 'Open Time', 'is_up', 'is_weekly_up']]

# Display the final DataFrame
print(final_df)


            Day           Open Time  is_up  is_weekly_up
0        Sunday 2019-09-08 17:55:00  False          True
1        Sunday 2019-09-08 18:00:00  False          True
2        Sunday 2019-09-08 18:05:00  False          True
3        Sunday 2019-09-08 18:10:00  False          True
4        Sunday 2019-09-08 18:15:00  False          True
...         ...                 ...    ...           ...
539094  Tuesday 2024-10-22 16:10:00   True         False
539095  Tuesday 2024-10-22 16:15:00  False         False
539096  Tuesday 2024-10-22 16:20:00  False         False
539097  Tuesday 2024-10-22 16:25:00  False         False
539098  Tuesday 2024-10-22 16:30:00   True         False

[539099 rows x 4 columns]


In [7]:
# Create a new column that represents only the time of the day (HH:MM)
combined_df['Time'] = combined_df['Open Time'].dt.time

# Group by 'Day' and 'Time' (for example, Monday 00:00, Monday 00:05)
grouped = combined_df.groupby(['Day', 'Time'])

# Initialize a dictionary to store the results
correlation_results = []

# Iterate over each group and calculate the correlation and matching percentage
for (day, time), group in grouped:
    # Calculate correlation between 'is_up' and 'is_weekly_up' (convert boolean to integer for correlation)
    is_up_values = group['is_up'].astype(int)
    is_weekly_up_values = group['is_weekly_up'].astype(int)

    if len(is_up_values) > 1:  # Need at least 2 values for correlation
        correlation = is_up_values.corr(is_weekly_up_values)
    else:
        correlation = None  # Correlation not meaningful for a single row

    # Calculate percentage of cases where 'is_up' and 'is_weekly_up' are the same
    matching_percentage = (is_up_values == is_weekly_up_values).mean() * 100

    # Append the results
    correlation_results.append({
        'Day': day,
        'Time': time,
        'Correlation': correlation,
        'Matching Percentage': matching_percentage
    })

# Convert results to a DataFrame
correlation_df = pd.DataFrame(correlation_results)

# Sort the DataFrame by 'Matching Percentage' in descending order
sorted_correlation_df = correlation_df.sort_values(by='Matching Percentage', ascending=False)

# Display the sorted results
print(sorted_correlation_df)


            Day      Time  Correlation  Matching Percentage
1654    Tuesday  17:50:00     0.234725            61.797753
105      Friday  08:45:00     0.190359            59.550562
1318   Thursday  13:50:00     0.187050            59.550562
796    Saturday  18:20:00     0.186280            59.550562
860    Saturday  23:40:00     0.182523            59.176030
...         ...       ...          ...                  ...
651    Saturday  06:15:00    -0.153924            42.322097
703    Saturday  10:35:00    -0.153202            42.322097
209      Friday  17:25:00    -0.159688            41.947566
2001  Wednesday  22:45:00    -0.170479            41.573034
1702    Tuesday  21:50:00    -0.204710            39.700375

[2016 rows x 4 columns]
